In [10]:
import os
import tensorflow as tf
import cv2
import numpy as np
from pathlib import Path
from mtcnn import MTCNN
from utils import load_annotations

def extract_face(image, image_size=(55, 47)):
    detector = MTCNN()
    faces = detector.detect_faces(image)
    
    if len(faces) > 0:
        x, y, width, height = faces[0]['box']
        x2, y2 = x + width, y + height
        face = image[y:y2, x:x2]
        face = cv2.resize(face, (image_size[1], image_size[0]))  # (width, height)
        return face
    return None

def load_faces(video_dir, save_dir, annotation, image_size=(55, 47), num_images=10):
    def generator():
        for video_name in os.listdir(video_dir):
            video_path = os.path.normpath(os.path.join(video_dir, video_name))
            
            if not os.path.isfile(video_path) or not video_name.lower().endswith(".mp4"):
                continue
            
            save_path = os.path.normpath(os.path.join(save_dir, Path(video_name).stem))
            os.makedirs(save_path, exist_ok=True)
            
            cap = cv2.VideoCapture(video_path)
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            interval = max(frame_count // num_images, 1)
            face_images = []
            
            for i in range(num_images):
                cap.set(cv2.CAP_PROP_POS_FRAMES, i * interval)
                ret, frame = cap.read()
                if not ret:
                    break
                face = extract_face(frame, image_size)
                if face is not None:
                    face_images.append(face)
                    
                    # Simpan wajah ke file JPG
                    face_filename = os.path.join(save_path, f'face_{i+1}.jpg')
                    cv2.imwrite(face_filename, face)
            
            cap.release()
            
            while len(face_images) < num_images:
                face_images.append(np.zeros((image_size[0], image_size[1], 3), dtype=np.float32))
            
            face_images = np.array(face_images[:num_images], dtype=np.float32)
            label = np.asarray(annotation.get(Path(video_name).stem, np.zeros(5)), dtype=np.float32)
            yield face_images, label
    
    dataset = tf.data.Dataset.from_generator(generator, output_signature=(
        tf.TensorSpec(shape=(10, image_size[0], image_size[1], 3), dtype=tf.float32),
        tf.TensorSpec(shape=(5,), dtype=tf.float32)
    ))
    return dataset

# List subset yang akan diproses
subsets = ["train", "val", "test"]

# Memuat semua anotasi sekaligus
annotation_train, annotation_val, annotation_test = load_annotations()
annotations = {"train": annotation_train, "val": annotation_val, "test": annotation_test}

for subset in subsets:
    video_dir = os.path.normpath(f"dataset/videos/{subset}")
    save_dir = os.path.normpath(f"dataset/videoface/{subset}")
    
    os.makedirs(save_dir, exist_ok=True)
    
    dataset = load_faces(video_dir, save_dir, annotations[subset]).batch(8)
    
    # Perbaikan kode penyimpanan dataset
    tf.data.Dataset.save(dataset, f"data/videoface/{subset}_ds")
    
    print(f"Dataset wajah {subset} disimpan dalam format tf.data di data/videoface/{subset}_ds")
    print(f"Wajah yang diekstrak dari video {subset} disimpan dalam folder {save_dir}")


Update annotation path !
1/1 [==============================] - 0s 193ms/step
Dataset wajah train disimpan dalam format tf.data di data/videoface/train_ds
Wajah yang diekstrak dari video train disimpan dalam folder dataset\videoface\train
1/1 [==============================] - 0s 248ms/step
Dataset wajah val disimpan dalam format tf.data di data/videoface/val_ds
Wajah yang diekstrak dari video val disimpan dalam folder dataset\videoface\val
1/1 [==============================] - 0s 300ms/step
Dataset wajah test disimpan dalam format tf.data di data/videoface/test_ds
Wajah yang diekstrak dari video test disimpan dalam folder dataset\videoface\test
